<a href="https://colab.research.google.com/github/priyadharshinirangasamy/Energy-efficiency-and-optimization-/blob/main/energy_efficiency_and_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# energy_efficiency_system.py

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from flask import Flask, request, jsonify
import threading
import time
import random
import requests
import joblib
import os

# 1. Generate sample energy dataset
def create_sample_data():
    np.random.seed(42)
    temperature = np.random.uniform(15, 35, 500)
    hour = np.random.randint(0, 24, 500)
    day_of_week = np.random.randint(0, 7, 500)
    # Energy = Base + temp_effect + time_effect + noise
    energy = 50 + (temperature * 1.2) + (hour * 0.8) + (day_of_week * 2) + np.random.normal(0, 5, 500)

    df = pd.DataFrame({
        'temperature': temperature,
        'hour': hour,
        'day_of_week': day_of_week,
        'energy_consumption': energy
    })
    df.to_csv('energy_data.csv', index=False)

# 2. Train and save the model
def train_model():
    data = pd.read_csv('energy_data.csv')
    X = data[['temperature', 'hour', 'day_of_week']]
    y = data['energy_consumption']
    model = LinearRegression()
    model.fit(X, y)
    joblib.dump(model, 'energy_predictor.pkl')

# 3. Flask API for real-time prediction
app = Flask(__name__)
model = None

@app.route('/predict', methods=['POST'])
def predict():
    global model
    content = request.json
    features = np.array([[content['temperature'], content['hour'], content['day_of_week']]])
    prediction = model.predict(features)
    return jsonify({'predicted_energy': round(prediction[0], 2)})

# 4. Simulated IoT device sending data to the API
def simulate_iot_device():
    time.sleep(5)  # Wait for server to start
    print("\n--- Simulated IoT Device Output ---\n")
    for i in range(5):  # Simulate 5 readings
        data = {
            'temperature': round(random.uniform(20.0, 35.0), 2),
            'hour': time.localtime().tm_hour,
            'day_of_week': time.localtime().tm_wday
        }
        try:
            response = requests.post('http://127.0.0.1:5000/predict', json=data)
            print(f"Sent: {data} → Predicted Energy: {response.json()['predicted_energy']}")
        except Exception as e:
            print("IoT Simulation Error:", e)
        time.sleep(3)

# 5. Run the server and simulate IoT together
if __name__ == '__main__':
    if not os.path.exists('energy_data.csv'):
        create_sample_data()
        train_model()

    model = joblib.load('energy_predictor.pkl')

    # Start IoT simulation in a separate thread
    threading.Thread(target=simulate_iot_device).start()

    print("Starting Energy Efficiency API on http://127.0.0.1:5000")
    app.run(debug=False)